In [125]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import base64
import hashlib
import unicodedata

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from typing import List, Dict
from typing import Callable
from datetime import datetime

In [126]:
# Funções de conversão
def remover_acentos_e_transformar_minusculo(texto):
    # Transforma o texto em minúsculas
    texto = texto.lower()
    # Normaliza a string para remover acentos
    texto_sem_acentos = unicodedata.normalize('NFD', texto)
    texto_sem_acentos = texto_sem_acentos.encode('ascii', 'ignore').decode('utf-8')
    return texto_sem_acentos

def str_null(df: pd.DataFrame, column: str) -> pd.DataFrame:
    cols_tela = [col for col in df.columns if col.startswith(column) and col not in [f'{column}_part_1'] ]    
    for column_name in cols_tela:
        df[column_name] = df[column_name].apply(lambda x: 1 if isinstance(x, str) else 0)        
    return df

def yes_no(df: pd.DataFrame, column: str) -> pd.DataFrame:
    cols_tela = [col for col in df.columns if col.startswith(column) and col not in [f'{column}_part_1'] ]    
    for column_name in cols_tela:
        df[column_name] = df[column_name].map({'Não': 0, 'Sim': 1, ' Não': 0, ' Sim': 1}).astype('int')
    return df

def one_hot_encode_column(df: pd.DataFrame, column_names: list[str]) -> pd.DataFrame:
    """
    Realiza One-Hot Encoding em uma coluna específica de um DataFrame, 
    inserindo as colunas codificadas imediatamente após a coluna original 
    e removendo a coluna original (de forma a evitar PerformanceWarning).

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame que será modificado (não é modificado in-place).
    column_names : List<str>
        Nomes das colunas em que o One-Hot Encoding será aplicado.

    Retorna
    -------
    pd.DataFrame
        Novo DataFrame modificado com colunas one-hot adicionadas.
    """
    for column_name in column_names:
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não existe no DataFrame.")

        # Cria o objeto OneHotEncoder
        enc = OneHotEncoder(sparse_output=False)

        # Seleciona a coluna como DataFrame (2D)
        X = df[[column_name]]

        # Gera array 2D encodado
        encoded_array = enc.fit_transform(X)

        # Nome das colunas encodadas
        encoded_cols = enc.get_feature_names_out([column_name])

        # Converte para DataFrame, mantendo o mesmo index
        encoded_df = pd.DataFrame(encoded_array, 
                                columns=encoded_cols, 
                                index=df.index)

        # Índice da coluna original
        col_index = df.columns.get_loc(column_name)

        # "Partes" do DataFrame:
        # - left_part = tudo antes da coluna alvo
        # - pivot_col = a própria coluna alvo
        # - right_part = tudo após a coluna alvo
        left_part = df.iloc[:, :col_index]
        pivot_col = df[[column_name]]
        right_part = df.iloc[:, col_index + 1:]

        # Concatena:
        #  1) tudo à esquerda
        #  2) a coluna original (pivot)
        #  3) as colunas encodadas
        #  4) tudo à direita
        # Depois removemos a coluna original
        temp_df = pd.concat([left_part, pivot_col, encoded_df, right_part], axis=1)

        # Remover a coluna original
        df = temp_df.drop(columns=[column_name])

    return df

# Caminho para o arquivo xlsx local
file_path = './banco_dados.xlsx'
# Carregar o DataFrame
df_inicial = pd.read_excel(file_path, sheet_name='BDados')
# Remover as colunas vazias
df_base = df_inicial.dropna(axis=1, how='all')

# Realizando várias substituições em todas as colunas do DataFrame
df = df_base.replace({
    "other;": "",   # Remove "other;"
    "Sucess;": "",  # Remove "Sucess;"
    "Sucess": ""    # Remove "Sucess"
}, regex=True)

In [127]:
# Função para dividir as colunas que têm valores separados por ponto e vírgula
def split_columns(df):
    columns_to_drop = []  # Lista para armazenar as colunas originais a serem removidas
    for col in df.columns:
        if df[col].dtype == 'object':  # Verificar se a coluna é do tipo string
            if df[col].str.contains(';').any():  # Se o valor é uma string com ";", quebra em novas colunas
                split_df = df[col].str.split(';', expand=True)
                split_df.columns = [f'{col}_part_{i+1}' for i in range(split_df.shape[1])]
                # Adicionar as novas colunas ao DataFrame
                df = pd.concat([df, split_df], axis=1)
                # Marcar a coluna original para ser removida
                columns_to_drop.append(col)
    
    # Remover as colunas originais
    df = df.drop(columns=columns_to_drop)
    return df

# Aplicar a função para dividir as colunas
df = split_columns(df)

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='XDados', index=False)

In [128]:
# Convertendo para datetime com formato brasileiro
def extract_timestamp(date_str): 
    return pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S.%f')
# Gerar a lista de colunas para aplicar a função
columns_to_apply  = [f'Tela {i+1:02}_part_1' for i in range(77)]
# Filtrar as colunas que realmente existem no DataFrame
existing_columns = [col for col in columns_to_apply if col in df.columns]
# Aplicar a função apenas nas colunas existentes
df[existing_columns] = df[existing_columns].map(extract_timestamp)

timeInicial = df['Tela 01_part_1']
# Subtrair a próxima coluna pela coluna anterior
for i in range(1, len(existing_columns)):
    df[existing_columns[i-1]] = (df[existing_columns[i]] - df[existing_columns[i-1]]).dt.total_seconds()
df['Tela 77_part_1'] = (df['Tela 77_part_1'] - timeInicial).dt.total_seconds() / 60

# Calcula o quanto o entrevistado estava errado em noção do dia em que ele esta
df['Tela 02_part_2'] = pd.to_datetime(df['Tela 02_part_3'] + df['Tela 02_part_2'],dayfirst=True, errors="coerce").sub(timeInicial).dt.total_seconds()
df = one_hot_encode_column(df,[
    'Tela 02_part_5',  # Gênero
    'Tela 02_part_6',  # Sexo do nascimento 
    'Tela 02_part_7',  # Cor ou Raça
    'Tela 02_part_9',  # Estado civil
    'Tela 02_part_12', # Religião
    'Tela 02_part_13', # Escolaridade
    'Tela 02_part_14', # Renda familiar
    ]) 
# Dentro de sua família, você é o(a) único(a) filho(a)?
df['Tela 02_part_8'] = df['Tela 02_part_8'].replace("Sim", 0).astype('int')
# Possui filhos(as)?
df['Tela 02_part_10'] = df['Tela 02_part_10'].replace("Não", 0).astype('int')
# Possui filhos(as) menores de 6 anos?
df['Tela 02_part_11'] = df['Tela 02_part_11'].map({'Não': 0, 'Sim': 1}).astype('int')
df = one_hot_encode_column(df,[
    'Tela 03_part_2', # Tela 03 - Target
    'Tela 07_part_2', # Tela 07 -                  
    'Tela 10_part_2', # Tela 10 -         
    'Tela 13_part_2', # Tela 13 -
    'Tela 15_part_2', # Tela 15 -
    'Tela 17_part_2', # Tela 17 -
    'Tela 19_part_2', # Tela 19 -
    'Tela 21_part_2', # Tela 21 -
    'Tela 23_part_2', # Tela 23 -
    ])
# Tela 25: # Dentre as quatro alternativas de cada imagem. Selecione a palavra que melhor a descreve
cols_tela = [col for col in df.columns if col.startswith('Tela 25_') and col not in ['Tela 25_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 26: # Responda as questões abaixo:
df = yes_no(df,'Tela 26')
# case 27: # Qual das imagens abaixo completa a sequência a seguir?
cols_tela = [col for col in df.columns if col.startswith('Tela 27_') and col not in ['Tela 27_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 28: # Responda as questões abaixo:
df = yes_no(df,'Tela 28')
# case 30: # Observe as palavras a seguir:
cols_tela = [col for col in df.columns if col.startswith('Tela 30_') and col not in ['Tela 30_part_1'] ]
df = one_hot_encode_column(df,cols_tela)          
# case 31: # Responda as questões abaixo: + 6
df = yes_no(df,'Tela 31')
# case 33: # Hora 
cols_tela = [col for col in df.columns if col.startswith('Tela 33_') and col not in ['Tela 33_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 41:
df = yes_no(df,'Tela 41')
# case 42:
def pontos_42(colData) -> int:
    index = 0.0
    for item in colData.split("|"):
        str = item.split(",")
        if 43.0 < float(str[0]) < 64.0 and 393.0 < float(str[1]) < 414.0: index += 1.0 #  53.799999999999955, 403.8 
        if 422.0 < float(str[0]) < 443.0 and 38.0 < float(str[1]) < 59.0: index += 1.0 # 432.79999999999995,   48.80000000000001;                
        if 28.0 < float(str[0]) < 49.0 and 201.0 < float(str[1]) < 222.0: index += 1.0 #  38.799999999999955, 211.8;                
        if 105.0 < float(str[0]) < 126.0 and 13.0 < float(str[1]) < 34.0: index += 1.0 # 115.79999999999995,   23.80000000000001;                
        if 274.0 < float(str[0]) < 295.0 and 369.0 < float(str[1]) < 390.0: index += 1.0 # 284.79999999999995,  379.8;                
        if 341.0 < float(str[0]) < 362.0 and 232.0 < float(str[1]) < 253.0: index += 1.0 # 351.79999999999995,  242.8                                                          
    return index
df['Tela 42_part_2'] = df['Tela 42_part_2'].apply(pontos_42)                                                                                 
# case 43:
cols_tela = [col for col in df.columns if col.startswith('Tela 43_') and col not in ['Tela 43_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 44: # + 6
df = yes_no(df,'Tela 44')
# case 47:
cols_tela = [col for col in df.columns if col.startswith('Tela 47_') and col not in ['Tela 47_part_1'] ]
df = one_hot_encode_column(df,cols_tela)    
# case 48:
cols_tela = [col for col in df.columns if col.startswith('Tela 48_') and col not in ['Tela 48_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 49:
df = one_hot_encode_column(df,['Tela 49_part_2']) 
# FUTURO df['Tela 49_part_3'] = df['Tela 49_part_3'] # Quando fazer aqui remove-lo da lista drop
# case 50: # + 7
df = yes_no(df,'Tela 50')
# case 51: # 
def pontos_51(colData) -> int:
    index = 0.0
    for item in colData.split("|"):
        str = item.split(",")
        if 196.0 < float(str[0]) < 217.0 and 579.0 < float(str[1]) < 600.0: index += 1.0 # 206.79999999999995,589.8 
        if 105.0 < float(str[0]) < 126.0 and 585.0 < float(str[1]) < 606.0: index += 1.0 # 115.79999999999995,595.8   
        if 206.0 < float(str[0]) < 227.0 and 46.0 < float(str[1]) < 67.0: index += 1.0 # 216.79999999999995,56.80000000000001
        if 376.0 < float(str[0]) < 397.0 and 480.0 < float(str[1]) < 501: index += 1.0 # 386.79999999999995,490.8
        if 293.0 < float(str[0]) < 314.0 and 668.0 < float(str[1]) < 689.0: index += 1.0 # 303.79999999999995,678.3169811320754
    return index
df['Tela 51_part_2'] = df['Tela 51_part_2'].apply(pontos_51)                 
# case 52: # + 7
df = yes_no(df,'Tela 52')
# case 53: # Qual das imagens abaixo completa a sequência a seguir?
df = str_null(df,'Tela 53')                    
# case 54: # + 7
df = yes_no(df,'Tela 54')  
# case 57: # Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
df = str_null(df,'Tela 57')              
# case 58: # + 7
df = yes_no(df,'Tela 58') 
# case 59: # Qual das opções corresponde ao som escutado?
cols_tela = [col for col in df.columns if col.startswith('Tela 59_') and col not in ['Tela 59_part_1'] ]
df = one_hot_encode_column(df,cols_tela)  
# case 60: # + 6 Preencha o campo a seguir com o nome da cidade e estado onde você está agora.
cols_tela = [col for col in df.columns if col.startswith('Tela 60_') and col not in ['Tela 60_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 62: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
cols_tela = [col for col in df.columns if col.startswith('Tela 62_') and col not in ['Tela 62_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 63: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
cols_tela = [col for col in df.columns if col.startswith('Tela 63_') and col not in ['Tela 63_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 64: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
cols_tela = [col for col in df.columns if col.startswith('Tela 64_') and col not in ['Tela 64_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 65: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
cols_tela = [col for col in df.columns if col.startswith('Tela 65_') and col not in ['Tela 65_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 66: # Selecione pelo menos 2 e no máximo 4 expressões que melhor correspondem ao que você tem sentido nos últimos meses.
cols_tela = [col for col in df.columns if col.startswith('Tela 66_') and col not in ['Tela 66_part_1'] ]
df = one_hot_encode_column(df,cols_tela)                
# case 69: # + 10
cols_tela = [col for col in df.columns if col.startswith('Tela 69_') and col not in ['Tela 69_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 70: # + 8
df = yes_no(df,'Tela 70')
# case 71: # + 5
cols_tela = [col for col in df.columns if col.startswith('Tela 71_') and col not in ['Tela 71_part_1'] ]
df = one_hot_encode_column(df,cols_tela)
# case 72: # + 7
df = yes_no(df,'Tela 72')
# case 73: # Daltonismo é o termo usado para denominar a falta de sensibilidade
cols_tela = [col for col in df.columns if col.startswith('Tela 73_') and col not in ['Tela 73_part_1'] ]
df = one_hot_encode_column(df,cols_tela)          
# case 74: # 
# FUTURO: Quando fazer aqui remove-lo da lista drop
# case 75: # 
# FUTURO: Quando fazer aqui remove-lo da lista drop
# Tela 76:
day_of_week = timeInicial.dt.day_name(locale='pt_BR').apply(remover_acentos_e_transformar_minusculo)
df['Tela 76_part_2'] = df['Tela 76_part_2'].str.strip().apply(remover_acentos_e_transformar_minusculo)
df['Tela 76_part_2'] = (df['Tela 76_part_2'] == day_of_week).astype(int)  
# Tela 77: Calcula o quanto o entrevistado estava errado em noção do tempo gasto para resolver o questionário.
def classify_time(faixa: str, valor: int) -> int: # Criar a coluna de classificação
    if faixa == ' Mais que 1 hora' and valor > 70: return 0
    elif faixa == ' 5 minutos': 
        return 5 - valor
    elif faixa == ' 15 minutos': 
        return 15 - valor
    elif faixa == ' 30 minutos': 
        return 30 - valor
    elif faixa == ' 40 minutos': 
        return 40 - valor
    elif faixa == ' 60 minutos' or faixa == ' Mais que 1 hora': 
        return 60 - valor
df['Tela 77_part_2'] = df.apply(lambda row: classify_time(row['Tela 77_part_2'], row['Tela 77_part_1']), axis=1)    

# Tela 32_part_2 -> Cidade
# Tela 32_part_3 -> Estado

df = df.drop(columns=['Tela 32_part_2', 'Tela 32_part_3', 'Tela 49_part_3', 'Tela 74_part_2', 'Tela 75_part_2', 'key','Mac Address_part_1','Mac Address_part_2','Mac Address_part_3','Tela 01_part_2','Tela 02_part_3','Tela 77_part_1'])

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='TDados', index=False)